In [1]:
import findspark
findspark.init()
import pyspark
import pyarrow
from pyspark.sql import SQLContext
import os
import sys
from pyspark.sql import SparkSession

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
conf = pyspark.SparkConf()\
        .setAppName('spark_pipeline')\
        .setMaster('local')\
        .set('spark.driver.memory', '8g')\
        .set('spark.executor.memory', '8g')\
        .set('spark.executor.instances', 4)\
        .set('spark.executor.cores', 4)\
        .set('spark.driver.maxResultSize', '8g')\
        .set('spark.sql.shuffle.partitions', 100)\
        .set('spark.default.parallelism', 200)\
        .set('spark.sql.broadcastTimeout', 36000)\
        .set('spark.kryoserializer.buffer.max', '1024m')\
        .set('spark.sql.execution.arrow.enabled', 'false')\
        .set('spark.dynamicAllocation.enabled', "False")\
        .set('spark.port.maxRetries',30) 
sc = pyspark.SparkContext.getOrCreate(conf)
spark = pyspark.sql.SparkSession(sc)
sqlContext = SQLContext.getOrCreate(sc)    


C:\Users\Davide\anaconda3\envs\pyspark_env\Lib\site-packages\pyspark\sql\context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.ml.feature import StringIndexer,OneHotEncoder
from pyspark.sql import SQLContext, SparkSession, Row
from pyspark.mllib.stat import Statistics
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, MinMaxScaler, StandardScaler
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import when, count, isnull
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import lit
from pyspark.sql.functions import when, count, isnull
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.classification import RandomForestClassifier

from pyspark.ml.functions import vector_to_array

import pyspark.sql.functions as F

from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.linalg import Vectors

#spark = SparkSession.Builder().appName('DDAM_Project_west').getOrCreate()

In [3]:
df = spark.read.options(inferSchema = True, delimiter = ',', header = True).csv('../../Datasets/West_Incidents_Cleaned.csv') 

In [4]:
df=df.drop('_c0') #devo droppare queste colonne. 

In [5]:
df_class=df

## Working_weekend

In [6]:
num_col = [item[0] for item in df_class.dtypes if not item[1].startswith('string')]
num_col.remove("Working_Weekend")
print(num_col)

['Severity', 'Start_Lat', 'Start_Lng', 'Distance_mi', 'Temperature_F', 'Humidity_perc', 'Visibility_mi', 'Wind_Speed_mph', 'Precipitation_in', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'month', 'hour', 'Astronomical_Twilight']


In [7]:
'''# Definisco funzione le metriche 
def print_metrics_and_cf(predictions):
    # Select (prediction, true label) and compute test error
    evaluator = MulticlassClassificationEvaluator(labelCol="Working_Weekend"
                                              , predictionCol="prediction"
                                              , metricName="accuracy")

    accuracy = evaluator.evaluate(predictions)
    print("Accuracy :",accuracy)
    print("Test Error = %g" % (1.0 - accuracy))
    print('----------------------------')
    
    # metrics
    predictionAndLabels = predictions.select("prediction", "Working_Weekend").rdd.map(lambda x: (float(x[0]), float(x[1])))
    metrics = MulticlassMetrics(predictionAndLabels)
    auc=metrics.areaUnderROC
    print('----------',auc)

    list_avg=[]
    for i in range(2):
        precision = metrics.precision(label=float(i))  
        recall = metrics.recall(label=float(i))
        f1Score= metrics.fMeasure(label=float(i)) # need .0
        print("Precision for class ",i+1,": {:.2%}".format(precision))
        print("Recall for class ",i+1,": {:.2%}".format(recall))
        print("avg_F1-Score for class ",i+1,": {:.2%}".format(f1Score))
        list_avg.append(f1Score)
        print('----------------------------')
    sum_=0
    #print(list_avg)
    for elem in list_avg:
        sum_+=elem

    avg_f1=sum_/4
    print('----------------------')

    print("avg_F1-Score: {:.2%}".format(avg_f1))

    # Confusion Matrix

    labels = ["0", "1"]# DEVO DEFINIRE working e Weekend 
    _ = plt.figure(figsize=(7, 7))
    sns.heatmap(metrics.confusionMatrix().toArray(),
                cmap='viridis',
                annot=True,fmt='0',
                cbar=False, 
                xticklabels=labels, 
                yticklabels=labels)'''

'# Definisco funzione le metriche \ndef print_metrics_and_cf(predictions):\n    # Select (prediction, true label) and compute test error\n    evaluator = MulticlassClassificationEvaluator(labelCol="Working_Weekend"\n                                              , predictionCol="prediction"\n                                              , metricName="accuracy")\n\n    accuracy = evaluator.evaluate(predictions)\n    print("Accuracy :",accuracy)\n    print("Test Error = %g" % (1.0 - accuracy))\n    print(\'----------------------------\')\n    \n    # metrics\n    predictionAndLabels = predictions.select("prediction", "Working_Weekend").rdd.map(lambda x: (float(x[0]), float(x[1])))\n    metrics = MulticlassMetrics(predictionAndLabels)\n    auc=metrics.areaUnderROC\n    print(\'----------\',auc)\n\n    list_avg=[]\n    for i in range(2):\n        precision = metrics.precision(label=float(i))  \n        recall = metrics.recall(label=float(i))\n        f1Score= metrics.fMeasure(label=float(i)

In [8]:
def print_metrics_and_cf(predictions):
    # Select (prediction, true label) and compute test error
    evaluator = MulticlassClassificationEvaluator(labelCol="Working_Weekend"
                                              , predictionCol="prediction"
                                              , metricName="accuracy")

    accuracy = evaluator.evaluate(predictions)
    print("Accuracy :",accuracy)
    print("Test Error = %g" % (1.0 - accuracy))
    print('----------------------------')

    # AUC score 
    scoreAndLabels = predictions.select('probability','Working_Weekend').rdd.map(lambda row: (float(row['Working_Weekend']),float(row['probability'][1]) ))
    metrics = BinaryClassificationMetrics(scoreAndLabels)
    auROC = metrics.areaUnderROC
    print('Auc_:', auROC,'\n ----------------------')
    
    
    # metrics
    predictionAndLabels = predictions.select("prediction", "Working_Weekend").rdd.map(lambda x: (float(x[0]), float(x[1])))
    metrics = MulticlassMetrics(predictionAndLabels)

    list_avg=[]
    for i in range(2):
        precision = metrics.precision(label=float(i))  
        recall = metrics.recall(label=float(i))
        f1Score= metrics.fMeasure(label=float(i)) # need .0
        print("Precision for class ",i+1,": {:.2%}".format(precision))
        print("Recall for class ",i+1,": {:.2%}".format(recall))
        print("avg_F1-Score for class ",i+1,": {:.2%}".format(f1Score))
        list_avg.append(f1Score)
        print('----------------------------')
    sum_=0
    #print(list_avg)
    for elem in list_avg:
        sum_+=elem

    avg_f1=sum_/2
    print('----------------------')

    print("avg_F1-Score: {:.2%}".format(avg_f1))

    # Confusion Matrix
    #alternativa per pc 
    print('----------------------')
    cf= metrics.confusionMatrix().toArray()
    print(cf)
    print('----------------------')
'''    labels = ["0", "1"]
    _ = plt.figure(figsize=(7, 7))
    sns.heatmap(metrics.confusionMatrix().toArray(),
                cmap='viridis',
                annot=True,fmt='0',
                cbar=False, 
                xticklabels=labels, 
                yticklabels=labels)'''

'    labels = ["0", "1"]\n    _ = plt.figure(figsize=(7, 7))\n    sns.heatmap(metrics.confusionMatrix().toArray(),\n                cmap=\'viridis\',\n                annot=True,fmt=\'0\',\n                cbar=False, \n                xticklabels=labels, \n                yticklabels=labels)'

### A_Sbilanciato

In [9]:
assembler = VectorAssembler(inputCols=num_col, outputCol="features")

output_dataset = assembler.transform(df_class)

classificationData = output_dataset.select("features", "Working_Weekend")

classificationData.show(truncate=False)

+---------------------------------------------------------------------------------------------------------+---------------+
|features                                                                                                 |Working_Weekend|
+---------------------------------------------------------------------------------------------------------+---------------+
|(24,[0,1,2,3,4,5,6,7,21,22],[2.0,36.980083,-120.04043,0.158,56.0,69.0,10.0,5.0,5.0,5.0])                 |0              |
|(24,[0,1,2,3,4,5,6,21,22],[2.0,37.02114701253405,-120.13138890209136,1.899,67.0,47.0,10.0,5.0,8.0])      |0              |
|(24,[0,1,2,3,4,5,6,7,21,22,23],[2.0,36.841894,-119.983215,0.644,60.0,39.0,10.0,6.0,5.0,23.0,1.0])        |0              |
|(24,[0,1,2,3,4,5,6,7,21,22,23],[2.0,36.851273,-119.992134,0.502,60.0,39.0,10.0,6.0,5.0,23.0,1.0])        |0              |
|(24,[0,1,2,3,4,5,6,7,18,21,22,23],[2.0,36.923019,-119.946995,0.409,73.0,23.0,10.0,8.0,1.0,5.0,22.0,1.0]) |1              |
|(24,[0,

In [10]:
#split in train and test 
(trainingData, testData) = classificationData.randomSplit([0.7, 0.3],seed=0)

In [11]:
from pyspark.sql import functions as F
g=trainingData.groupBy('Working_Weekend').agg(F.count('Working_Weekend').alias('Count'))
g=g.sort('Working_Weekend')
g.show()

+---------------+------+
|Working_Weekend| Count|
+---------------+------+
|              0|205925|
|              1| 61513|
+---------------+------+



In [12]:
# Save the count of instances
g_p=g.toPandas()
n_0=g_p.iloc[0,1]
n_1=g_p.iloc[1,1]

In [13]:
trainingData.show()

+--------------------+---------------+
|            features|Working_Weekend|
+--------------------+---------------+
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              1|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              1|
|(24,[0,1,2,3,4,5,...|              1|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
+--------------------+---------------+
only showing top 20 rows



In [14]:
testData.show()

+--------------------+---------------+
|            features|Working_Weekend|
+--------------------+---------------+
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              1|
|(24,[0,1,2,3,4,5,...|              1|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
|(24,[0,1,2,3,4,5,...|              0|
+--------------------+---------------+
only showing top 20 rows



### A_ DT base

In [15]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier

dt = DecisionTreeClassifier(labelCol="Working_Weekend", featuresCol="features")

dt = dt.fit(trainingData)

In [16]:
# Make predictions.
predictions = dt.transform(testData)

predictions.show(20)

+--------------------+---------------+------------------+--------------------+----------+
|            features|Working_Weekend|     rawPrediction|         probability|prediction|
+--------------------+---------------+------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|              0|[128036.0,38947.0]|[0.76676068821377...|       0.0|
|(24,[0,1,2,3,4,5,...|              0|    [7392.0,739.0]|[0.90911327020046...|       0.0|
|(24,[0,1,2,3,4,5,...|              0|[128036.0,38947.0]|[0.76676068821377...|       0.0|
|(24,[0,1,2,3,4,5,...|              0|   [11831.0,509.0]|[0.95875202593192...|       0.0|
|(24,[0,1,2,3,4,5,...|              0|[128036.0,38947.0]|[0.76676068821377...|       0.0|
|(24,[0,1,2,3,4,5,...|              0|   [11831.0,509.0]|[0.95875202593192...|       0.0|
|(24,[0,1,2,3,4,5,...|              0|  [27415.0,4613.0]|[0.85596977644561...|       0.0|
|(24,[0,1,2,3,4,5,...|              0|[128036.0,38947.0]|[0.76676068821377...|       0.0|
|(24,[0,1,

In [18]:
print_metrics_and_cf(predictions)

Accuracy : 0.7709629031833612
Test Error = 0.229037
----------------------------
Auc_: 0.6820996993299244 
 ----------------------
Precision for class  1 : 77.18%
Recall for class  1 : 99.77%
avg_F1-Score for class  1 : 87.03%
----------------------------
Precision for class  2 : 59.24%
Recall for class  2 : 1.14%
avg_F1-Score for class  2 : 2.23%
----------------------------
----------------------
avg_F1-Score: 44.63%
----------------------
[[87591.   205.]
 [25905.   298.]]
----------------------


In [19]:
dt.featureImportances

SparseVector(24, {0: 0.0173, 1: 0.0181, 3: 0.1032, 8: 0.0271, 21: 0.0204, 22: 0.381, 23: 0.4329})

### A___ DT testo configurazioni diverse di parametri

#### a)

In [20]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier

dt_param1 = DecisionTreeClassifier(labelCol="Working_Weekend", featuresCol="features",
                                  maxDepth=20,
                                  maxBins=32)
                                   

dt_param1 = dt_param1.fit(trainingData)
predictions_param1 = dt_param1.transform(testData)
print_metrics_and_cf(predictions_param1)

Accuracy : 0.7876735760840007
Test Error = 0.212326
----------------------------
Auc_: 0.6938508275341404 
 ----------------------
Precision for class  1 : 82.14%
Recall for class  1 : 92.56%
avg_F1-Score for class  1 : 87.04%
----------------------------
Precision for class  2 : 56.63%
Recall for class  2 : 32.55%
avg_F1-Score for class  2 : 41.34%
----------------------------
----------------------
avg_F1-Score: 64.19%
----------------------
[[81266.  6530.]
 [17675.  8528.]]
----------------------


In [21]:
dt_param1.featureImportances

SparseVector(24, {0: 0.0138, 1: 0.1427, 2: 0.1205, 3: 0.101, 4: 0.1299, 5: 0.1168, 6: 0.0416, 7: 0.0832, 8: 0.019, 9: 0.0016, 10: 0.0004, 11: 0.0046, 12: 0.0006, 13: 0.0058, 14: 0.0004, 15: 0.0022, 17: 0.0022, 18: 0.0041, 19: 0.0001, 20: 0.0032, 21: 0.0815, 22: 0.0869, 23: 0.0379})

#### b)


In [22]:
n=trainingData.count()

In [23]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier

dt_param1 = DecisionTreeClassifier(labelCol="Working_Weekend", featuresCol="features",
                                  maxDepth=30,
                                  maxBins=32,
                                  #minInstancesPerNode=round(0.01*n)
                                  )
                                   

dt_param1 = dt_param1.fit(trainingData)
predictions_param1 = dt_param1.transform(testData)
print_metrics_and_cf(predictions_param1)

Accuracy : 0.7891735892420109
Test Error = 0.210826
----------------------------
Auc_: 0.7012884398417275 
 ----------------------
Precision for class  1 : 85.99%
Recall for class  1 : 86.76%
avg_F1-Score for class  1 : 86.37%
----------------------------
Precision for class  2 : 54.27%
Recall for class  2 : 52.64%
avg_F1-Score for class  2 : 53.44%
----------------------------
----------------------
avg_F1-Score: 69.91%
----------------------
[[76171. 11625.]
 [12409. 13794.]]
----------------------


In [24]:
dt_param1.featureImportances

SparseVector(24, {0: 0.0103, 1: 0.1587, 2: 0.1342, 3: 0.1159, 4: 0.1375, 5: 0.1269, 6: 0.032, 7: 0.0896, 8: 0.0104, 9: 0.0019, 10: 0.0003, 11: 0.0054, 12: 0.0005, 13: 0.0075, 14: 0.0004, 15: 0.0017, 17: 0.0028, 18: 0.0055, 19: 0.0001, 20: 0.0046, 21: 0.0746, 22: 0.0601, 23: 0.0191})

## SOPRA RISULTATO MIGLIORE

#### c)

In [25]:
from pyspark.sql import functions as F
g=trainingData.groupBy('Working_Weekend').agg(F.count('Working_Weekend').alias('Count'))
g.sort('Count').show()

+---------------+------+
|Working_Weekend| Count|
+---------------+------+
|              1| 61513|
|              0|205925|
+---------------+------+



In [26]:
#trainingData_w = trainingData.withColumn("weights1", lit(1)) # questo è con tutti pesi 1  
# Creiamo la colonna 'weights' basata sulla frequenza delle classi
#trainingData_w
trainingData_w = trainingData.withColumn("weights2", when(trainingData["Working_Weekend"] == 0, 1.0)
    .when(trainingData["Working_Weekend"] == 1, 267020 / 61237)  # Calcolo del peso per la classe 1
)

#### c_jj) provo la lista dei pesi weights2

In [27]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier

dt_param1 = DecisionTreeClassifier(labelCol="Working_Weekend", featuresCol="features",
                                  maxDepth=30,
                                  maxBins=32,
                                  #Thresholds=[0.60,0.40]
                                weightCol='weights2'  
                                  )
                                   

dt_param1 = dt_param1.fit(trainingData_w)
predictions_param1 = dt_param1.transform(testData)
print_metrics_and_cf(predictions_param1)

Accuracy : 0.7684277932262563
Test Error = 0.231572
----------------------------
Auc_: 0.6829655813343057 
 ----------------------
Precision for class  1 : 86.91%
Recall for class  1 : 82.33%
avg_F1-Score for class  1 : 84.56%
----------------------------
Precision for class  2 : 49.68%
Recall for class  2 : 58.46%
avg_F1-Score for class  2 : 53.71%
----------------------------
----------------------
avg_F1-Score: 69.14%
----------------------
[[72283. 15513.]
 [10886. 15317.]]
----------------------


In [28]:
dt_param1.featureImportances

SparseVector(24, {0: 0.0185, 1: 0.1397, 2: 0.1252, 3: 0.1154, 4: 0.1325, 5: 0.1273, 6: 0.0324, 7: 0.0915, 8: 0.008, 9: 0.0018, 10: 0.0001, 11: 0.006, 12: 0.0004, 13: 0.0086, 14: 0.0004, 15: 0.0013, 17: 0.0027, 18: 0.0048, 19: 0.0001, 20: 0.0057, 21: 0.0771, 22: 0.0777, 23: 0.0228})

### B_Random Forest

In [29]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="Working_Weekend", featuresCol="features", numTrees=10)


# Train model.  This also runs the indexers.
model = rf.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)
print_metrics_and_cf(predictions)

Accuracy : 0.7701471065535662
Test Error = 0.229853
----------------------------
Auc_: 0.0 
 ----------------------
Precision for class  1 : 77.01%
Recall for class  1 : 100.00%
avg_F1-Score for class  1 : 87.02%
----------------------------
Precision for class  2 : 0.00%
Recall for class  2 : 0.00%
avg_F1-Score for class  2 : 0.00%
----------------------------
----------------------
avg_F1-Score: 43.51%
----------------------
[[87796.     0.]
 [26203.     0.]]
----------------------


In [30]:
model.featureImportances

SparseVector(24, {0: 0.1005, 1: 0.0204, 2: 0.0028, 3: 0.1468, 4: 0.0117, 5: 0.0038, 6: 0.0054, 7: 0.0021, 8: 0.0099, 14: 0.0001, 15: 0.0002, 17: 0.0, 18: 0.0006, 20: 0.0003, 21: 0.0132, 22: 0.4612, 23: 0.2211})

### B_i) TUNING RANDOM FOREST

In [31]:
rf = RandomForestClassifier(labelCol="Working_Weekend", featuresCol="features", numTrees=20,maxDepth=20, seed=10
                           )
# Train model.  This also runs the indexers.
model = rf.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)
print_metrics_and_cf(predictions)

Accuracy : 0.7965859349643418
Test Error = 0.203414
----------------------------
Auc_: 0.801283846214784 
 ----------------------
Precision for class  1 : 79.61%
Recall for class  1 : 98.92%
avg_F1-Score for class  1 : 88.22%
----------------------------
Precision for class  2 : 80.64%
Recall for class  2 : 15.14%
avg_F1-Score for class  2 : 25.49%
----------------------------
----------------------
avg_F1-Score: 56.85%
----------------------
[[86844.   952.]
 [22237.  3966.]]
----------------------


In [32]:
model.featureImportances

SparseVector(24, {0: 0.0214, 1: 0.1028, 2: 0.098, 3: 0.0979, 4: 0.1018, 5: 0.1024, 6: 0.0456, 7: 0.0763, 8: 0.0251, 9: 0.0028, 10: 0.0003, 11: 0.0068, 12: 0.0007, 13: 0.0092, 14: 0.0009, 15: 0.003, 17: 0.0047, 18: 0.0064, 19: 0.0007, 20: 0.0074, 21: 0.0895, 22: 0.1527, 23: 0.0437})

### C_NN

In [33]:
df = spark.read.options(inferSchema = True, delimiter = ',', header = True).csv('WC_NoMissing_OK.csv') 
df=df.drop('Wind_Chill_F','Pressure_in','_c0') #devo droppare queste colonne. 

In [34]:
to_drop=['Start_Time','City','County','State','Wind_Direction','day_of_the_week','season']
df_class=df.drop(*to_drop)
colonne_booleane = ['Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal']

# Converti le colonne booleane in numeriche
for col_name in colonne_booleane:
    df_class = df_class.withColumn(col_name, col(col_name).cast("int"))


df_class = df_class.withColumn("Astronomical_Twilight", when(df_class["Astronomical_Twilight"] == 'Day', 0).otherwise(1))
df_class = df_class.withColumn("Working_Weekend", when(df_class["Working_Weekend"] == 'WorkingDay', 0).otherwise(1))




indexer = StringIndexer(inputCol='Weather_Condition', outputCol='class_numeric')
indexer_fitted = indexer.fit(df_class)
df_indexed = indexer_fitted.transform(df_class)


encoder = OneHotEncoder(inputCols=['class_numeric'], outputCols=['class_onehot'],dropLast=False)
df_onehot = encoder.fit(df_indexed).transform(df_indexed)


df_col_onehot = df_onehot.select('*', vector_to_array('class_onehot').alias('col_onehot'))

num_categories = len(df_col_onehot.first()['col_onehot'])   # 3
cols_expanded = [(F.col('col_onehot')[i].alias(f'{indexer_fitted.labels[i]}')) for i in range(num_categories)]
df_cols_onehot = df_col_onehot.select('Severity',
 'Start_Lat',
 'Start_Lng',
 'Distance_mi',
 'Temperature_F',
 'Humidity_perc',
 'Visibility_mi',
 'Wind_Speed_mph',
 'Precipitation_in',
 'Amenity',
 'Bump',
 'Crossing',
 'Give_Way',
 'Junction',
 'No_Exit',
 'Railway',
 'Roundabout',
 'Station',
 'Stop',
 'Traffic_Calming',
 'Traffic_Signal',
 'month',
 'hour',
 'Working_Weekend',
 'Astronomical_Twilight', *cols_expanded)


num_col = [item[0] for item in df_cols_onehot.dtypes if not item[1].startswith('string')]
num_col.remove("Severity")

num_col_ww=[item[0] for item in df_cols_onehot.dtypes if not item[1].startswith('string')]
num_col_ww.remove("Working_Weekend")

df_MLP= df_cols_onehot.withColumn("Severity", col("Severity") - 1)
#df_MLP.show()

In [35]:
df_MLP_binary=df_MLP

In [36]:
assembler_NN = VectorAssembler(inputCols=num_col_ww, outputCol="features")

output_dataset_NN = assembler_NN.transform(df_MLP_binary)

classificationData_NN = output_dataset_NN.select("features", "Working_Weekend")

classificationData_NN.show(truncate=False)
(trainingData_nn, testData_nn) = classificationData_NN.randomSplit([0.7, 0.3],seed=0)

+------------------------------------------------------------------------------------------------------------------+---------------+
|features                                                                                                          |Working_Weekend|
+------------------------------------------------------------------------------------------------------------------+---------------+
|(29,[0,1,2,3,4,5,6,7,18,21,22,24],[1.0,39.324703,-123.802754,0.126,68.0,35.0,10.0,10.0,1.0,5.0,11.0,1.0])         |0              |
|(29,[0,1,2,3,4,5,6,7,21,22,24],[1.0,38.991387,-123.353783,0.513,73.0,27.0,10.0,12.0,5.0,16.0,1.0])                |0              |
|(29,[0,1,2,3,4,5,6,7,21,22,24],[1.0,38.98485,-123.350998,0.935,73.0,27.0,10.0,12.0,5.0,16.0,1.0])                 |0              |
|(29,[0,1,2,3,4,5,6,7,21,22,24],[1.0,39.771345,-123.248122,0.64,83.0,18.0,10.0,7.0,5.0,16.0,1.0])                  |0              |
|(29,[0,1,2,3,4,5,6,7,21,22,24],[1.0,39.814453,-123.065282,1.267,50.0

In [37]:
# MLP Layers  -> output layer must have the same number of units of the Working_Weekend classes

layers = [len(num_col_ww),16,8,4,2]

# Create the Multilayer Perceptron Classifier and set its parameters
trainer = MultilayerPerceptronClassifier(
    layers=layers,
    labelCol="Working_Weekend",
    featuresCol="features",
    maxIter=50, #!!!!!!!!!!!!!  
    tol=1e-6,
    seed=None,
    blockSize=32,
    stepSize=0.03,  
    solver="l-bfgs",
    initialWeights=None,
    probabilityCol="probability",
    rawPredictionCol="rawPrediction"
)

# Train the model
model = trainer.fit(trainingData_nn)

# Make predictions on the test set
predictions= model.transform(testData_nn)

print_metrics_and_cf(predictions)

Accuracy : 0.7686605652147309
Test Error = 0.231339
----------------------------
Auc_: 0.0 
 ----------------------
Precision for class  1 : 76.87%
Recall for class  1 : 100.00%
avg_F1-Score for class  1 : 86.92%
----------------------------
Precision for class  2 : 0.00%
Recall for class  2 : 0.00%
avg_F1-Score for class  2 : 0.00%
----------------------------
----------------------
avg_F1-Score: 43.46%
----------------------
[[87934.     0.]
 [26465.     0.]]
----------------------


In [38]:
#As we can see from the results accuracy is very high because the model predict only the labels 1,
#cause of the unbalanced data. Now we have to sample the training set trying the define a model well trained 
#and that can predict each labels on the test set.

## UNDERSAMPLING

In [39]:
train_und = trainingData.sampleBy('Working_Weekend', fractions={0:n_1/n_0 ,1: 1})

### U_A) DT base

In [40]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(labelCol="Working_Weekend", featuresCol="features")

dt = dt.fit(train_und)
predictions = dt.transform(testData)
print_metrics_and_cf(predictions)

Accuracy : 0.6284002491249924
Test Error = 0.3716
----------------------------
Auc_: 0.579449933011744 
 ----------------------
Precision for class  1 : 83.43%
Recall for class  1 : 64.57%
avg_F1-Score for class  1 : 72.80%
----------------------------
Precision for class  2 : 32.46%
Recall for class  2 : 57.04%
avg_F1-Score for class  2 : 41.37%
----------------------------
----------------------
avg_F1-Score: 57.09%
----------------------
[[56690. 31106.]
 [11256. 14947.]]
----------------------


In [41]:
dt.featureImportances

SparseVector(24, {0: 0.0384, 1: 0.0099, 3: 0.1458, 4: 0.0026, 7: 0.0024, 22: 0.5192, 23: 0.2816})

### U_A)_a) DT tuning 

In [42]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier

dt_param1 = DecisionTreeClassifier(labelCol="Working_Weekend", featuresCol="features",
                                  maxDepth=20,
                                  maxBins=32)
                                   

dt_param1 = dt_param1.fit(train_und)
predictions_param1 = dt_param1.transform(testData)
print_metrics_and_cf(predictions_param1)

Accuracy : 0.6443038974026088
Test Error = 0.355696
----------------------------
Auc_: 0.6147521938160576 
 ----------------------
Precision for class  1 : 87.17%
Recall for class  1 : 63.10%
avg_F1-Score for class  1 : 73.21%
----------------------------
Precision for class  2 : 35.78%
Recall for class  2 : 68.88%
avg_F1-Score for class  2 : 47.10%
----------------------------
----------------------
avg_F1-Score: 60.15%
----------------------
[[55401. 32395.]
 [ 8154. 18049.]]
----------------------


In [43]:
dt_param1.featureImportances

SparseVector(24, {0: 0.0239, 1: 0.1377, 2: 0.1139, 3: 0.1143, 4: 0.1187, 5: 0.1118, 6: 0.0354, 7: 0.0756, 8: 0.0137, 9: 0.0017, 10: 0.0001, 11: 0.0045, 12: 0.0004, 13: 0.0068, 14: 0.0003, 15: 0.0017, 16: 0.0, 17: 0.0025, 18: 0.0035, 19: 0.0, 20: 0.0038, 21: 0.0805, 22: 0.1096, 23: 0.0397})

### U_A)_b) DT tuning aumento numero profondità dell'albero e numero di bins

In [44]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier

dt_param1 = DecisionTreeClassifier(labelCol="Working_Weekend", featuresCol="features",
                                  maxDepth=30,
                                  maxBins=64)
                                   

dt_param1 = dt_param1.fit(train_und)
predictions_param1 = dt_param1.transform(testData)
print_metrics_and_cf(predictions_param1)

Accuracy : 0.6755322415108904
Test Error = 0.324468
----------------------------
Auc_: 0.6338804140436559 
 ----------------------
Precision for class  1 : 88.15%
Recall for class  1 : 66.86%
avg_F1-Score for class  1 : 76.04%
----------------------------
Precision for class  2 : 38.63%
Recall for class  2 : 69.89%
avg_F1-Score for class  2 : 49.75%
----------------------------
----------------------
avg_F1-Score: 62.90%
----------------------
[[58697. 29099.]
 [ 7890. 18313.]]
----------------------


In [45]:
dt_param1.featureImportances

SparseVector(24, {0: 0.0158, 1: 0.1797, 2: 0.1302, 3: 0.1274, 4: 0.1317, 5: 0.1243, 6: 0.0256, 7: 0.0708, 8: 0.0076, 9: 0.0015, 10: 0.0001, 11: 0.0049, 12: 0.0002, 13: 0.0069, 14: 0.0003, 15: 0.0013, 17: 0.0022, 18: 0.0044, 19: 0.0001, 20: 0.0044, 21: 0.0685, 22: 0.0699, 23: 0.0222})

### U_A)_c) DT tuning con pesi 


In [46]:
from pyspark.sql import functions as F
g=train_und.groupBy('Working_Weekend').agg(F.count('Working_Weekend').alias('Count'))
g.sort('Count').show()

+---------------+-----+
|Working_Weekend|Count|
+---------------+-----+
|              1|61513|
|              0|61572|
+---------------+-----+



In [47]:
trainingData_w = trainingData_w.withColumn("weights2", when(trainingData["Working_Weekend"] == 0, 1.0)
    .when(trainingData["Working_Weekend"] == 1, 122526 / 61237)  # Calcolo del peso per la classe 1
)

In [48]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier

dt_param2 = DecisionTreeClassifier(labelCol="Working_Weekend", featuresCol="features",
                                  maxDepth=30,
                                  maxBins=32,
                                weightCol='weights2'  
                                  )
                                   

dt_param2 = dt_param2.fit(trainingData_w)
predictions_param2 = dt_param2.transform(testData)
print_metrics_and_cf(predictions_param2)

Accuracy : 0.7873928718672971
Test Error = 0.212607
----------------------------
Auc_: 0.7010528464525068 
 ----------------------
Precision for class  1 : 86.59%
Recall for class  1 : 85.66%
avg_F1-Score for class  1 : 86.12%
----------------------------
Precision for class  2 : 53.62%
Recall for class  2 : 55.55%
avg_F1-Score for class  2 : 54.57%
----------------------------
----------------------
avg_F1-Score: 70.34%
----------------------
[[75207. 12589.]
 [11648. 14555.]]
----------------------


In [49]:
dt_param2.featureImportances

SparseVector(24, {0: 0.0116, 1: 0.1459, 2: 0.1259, 3: 0.1172, 4: 0.1398, 5: 0.1274, 6: 0.0325, 7: 0.0903, 8: 0.0083, 9: 0.0014, 10: 0.0003, 11: 0.0055, 12: 0.0004, 13: 0.0085, 14: 0.0004, 15: 0.0014, 17: 0.0028, 18: 0.005, 19: 0.0002, 20: 0.0048, 21: 0.0736, 22: 0.0739, 23: 0.023})

## il miglior DT

### U_B) Random Forest Base

In [50]:
# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="Working_Weekend", featuresCol="features", numTrees=10)


# Train model.  This also runs the indexers.
model = rf.fit(train_und)

# Make predictions.
predictions = model.transform(testData)
print_metrics_and_cf(predictions)

Accuracy : 0.658768936569619
Test Error = 0.341231
----------------------------
Auc_: 0.573835071731319 
 ----------------------
Precision for class  1 : 81.83%
Recall for class  1 : 71.58%
avg_F1-Score for class  1 : 76.37%
----------------------------
Precision for class  2 : 32.93%
Recall for class  2 : 46.75%
avg_F1-Score for class  2 : 38.65%
----------------------------
----------------------
avg_F1-Score: 57.51%
----------------------
[[62848. 24948.]
 [13952. 12251.]]
----------------------


In [51]:
model.featureImportances

SparseVector(24, {0: 0.0598, 1: 0.0149, 2: 0.0099, 3: 0.1754, 4: 0.0055, 5: 0.0094, 6: 0.0086, 7: 0.0091, 8: 0.0249, 11: 0.0001, 13: 0.0002, 16: 0.0, 17: 0.0001, 18: 0.0001, 20: 0.0001, 21: 0.0206, 22: 0.4004, 23: 0.2607})

### U_B)_a) Ranodom Forest Tuning aumento max depth e numero di alberi

In [52]:
rf = RandomForestClassifier(labelCol="Working_Weekend", featuresCol="features", numTrees=20,maxDepth=20, seed=10
                           )
# Train model.  This also runs the indexers.
model = rf.fit(train_und)

# Make predictions.
predictions = model.transform(testData)
print_metrics_and_cf(predictions)

Accuracy : 0.6794445565311976
Test Error = 0.320555
----------------------------
Auc_: 0.6455109670992571 
 ----------------------
Precision for class  1 : 89.59%
Recall for class  1 : 66.05%
avg_F1-Score for class  1 : 76.04%
----------------------------
Precision for class  2 : 39.51%
Recall for class  2 : 74.30%
avg_F1-Score for class  2 : 51.59%
----------------------------
----------------------
avg_F1-Score: 63.81%
----------------------
[[57988. 29808.]
 [ 6735. 19468.]]
----------------------


In [53]:
model.featureImportances

SparseVector(24, {0: 0.0286, 1: 0.0989, 2: 0.0943, 3: 0.1136, 4: 0.0982, 5: 0.1005, 6: 0.041, 7: 0.0736, 8: 0.0183, 9: 0.0026, 10: 0.0002, 11: 0.0067, 12: 0.0006, 13: 0.0096, 14: 0.0005, 15: 0.0024, 16: 0.0, 17: 0.0042, 18: 0.006, 19: 0.0004, 20: 0.0076, 21: 0.0885, 22: 0.1654, 23: 0.038})

### U_C) NN

In [54]:
g=trainingData_nn.groupBy('Working_Weekend').agg(F.count('Working_Weekend').alias('Count'))
g=g.sort('Working_Weekend')
g.show()

g_p=g.toPandas()
n_0=g_p.iloc[0,1]
n_1=g_p.iloc[1,1]

+---------------+------+
|Working_Weekend| Count|
+---------------+------+
|              0|205787|
|              1| 61251|
+---------------+------+



In [55]:
train_und_nn = trainingData_nn.sampleBy('Working_Weekend', fractions={0:n_1/n_0 ,1: 1})

In [56]:
layers = [len(num_col),4]

# Create the Multilayer Perceptron Classifier and set its parameters
trainer = MultilayerPerceptronClassifier(
    layers=layers,
    labelCol="Working_Weekend",
    featuresCol="features",
    maxIter=100,  
    tol=1e-6,
    seed=None,
    blockSize=32,
    stepSize=0.03,  
    solver="l-bfgs",
    initialWeights=None,
    probabilityCol="probability",
    rawPredictionCol="rawPrediction"
)

# Train the model
model = trainer.fit(train_und_nn)

# Make predictions on the test set
predictions= model.transform(testData_nn)
print_metrics_and_cf(predictions)

Accuracy : 0.6377066233096443
Test Error = 0.362293
----------------------------


C:\Users\Davide\anaconda3\envs\pyspark_env\Lib\site-packages\pyspark\sql\context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Auc_: 0.5483206328588364 
 ----------------------
Precision for class  1 : 79.98%
Recall for class  1 : 70.52%
avg_F1-Score for class  1 : 74.95%
----------------------------
Precision for class  2 : 29.68%
Recall for class  2 : 41.35%
avg_F1-Score for class  2 : 34.56%
----------------------------
----------------------
avg_F1-Score: 54.76%
----------------------
[[62009. 25925.]
 [15521. 10944.]]
----------------------


### OVERSAMPLING MINORITY CLASS

In [57]:
under_1=trainingData.filter(col('Working_Weekend') == 1)
under_1 = trainingData.sample(True, 0.3, seed=42)

In [58]:
under_1.count()

80137

In [59]:
oversample_1=trainingData.filter(col('Working_Weekend') == 1)
num_campioni_da_generare = n_0
num_campioni_da_generare=(num_campioni_da_generare-n_1)/n_1 #percentuale 
#print(num_campioni_da_generare)
full_1=oversample_1
#print(full.count())
while num_campioni_da_generare>1:
    oversample_1= oversample_1.unionAll(full_1)
    num_campioni_da_generare=num_campioni_da_generare-1.0
    print(num_campioni_da_generare)
df_minority_oversampled = full_1.sample(True, num_campioni_da_generare, seed=42)

oversample_1 = oversample_1.unionAll(df_minority_oversampled)

1.359732902319962
0.359732902319962


In [60]:
df_over=oversample_1.unionAll(trainingData.filter(col('Working_Weekend') == 0))

In [61]:
df_over.count()

412576

In [62]:
train_over=df_over

In [63]:
from pyspark.sql import functions as F
g=df_over.groupBy('Working_Weekend').agg(F.count('Working_Weekend').alias('Working_Weekend'))
g=g.sort('Working_Weekend')
g.show()

+---------------+---------------+
|Working_Weekend|Working_Weekend|
+---------------+---------------+
|              0|         205925|
|              1|         206651|
+---------------+---------------+



### O_A) DT Base

In [64]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(labelCol="Working_Weekend", featuresCol="features")

dt = dt.fit(train_over)

In [65]:
predictions = dt.transform(testData)
print_metrics_and_cf(predictions)

Accuracy : 0.6279704207931649
Test Error = 0.37203
----------------------------
Auc_: 0.580012434171895 
 ----------------------
Precision for class  1 : 83.51%
Recall for class  1 : 64.41%
avg_F1-Score for class  1 : 72.73%
----------------------------
Precision for class  2 : 32.49%
Recall for class  2 : 57.39%
avg_F1-Score for class  2 : 41.49%
----------------------------
----------------------
avg_F1-Score: 57.11%
----------------------
[[56550. 31246.]
 [11165. 15038.]]
----------------------


In [66]:
dt.featureImportances

SparseVector(24, {0: 0.0219, 1: 0.0198, 3: 0.1612, 4: 0.0035, 7: 0.0026, 18: 0.0004, 22: 0.5014, 23: 0.2891})

### O_A)_a) DT Tuning

In [67]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier

dt_param1 = DecisionTreeClassifier(labelCol="Working_Weekend", featuresCol="features",
                                  maxDepth=20,
                                  maxBins=32)
                                   

dt_param1 = dt_param1.fit(train_over)
predictions_param1 = dt_param1.transform(testData)
print_metrics_and_cf(predictions_param1)

Accuracy : 0.6752339932806428
Test Error = 0.324766
----------------------------
Auc_: 0.6282421611421578 
 ----------------------
Precision for class  1 : 87.39%
Recall for class  1 : 67.59%
avg_F1-Score for class  1 : 76.22%
----------------------------
Precision for class  2 : 38.26%
Recall for class  2 : 67.31%
avg_F1-Score for class  2 : 48.79%
----------------------------
----------------------
avg_F1-Score: 62.51%
----------------------
[[59339. 28457.]
 [ 8566. 17637.]]
----------------------


In [68]:
dt_param1.featureImportances

SparseVector(24, {0: 0.0215, 1: 0.1192, 2: 0.1087, 3: 0.1057, 4: 0.1257, 5: 0.1173, 6: 0.0388, 7: 0.0783, 8: 0.013, 9: 0.0014, 10: 0.0001, 11: 0.0041, 12: 0.0001, 13: 0.0052, 14: 0.0003, 15: 0.0016, 17: 0.0021, 18: 0.0036, 19: 0.0001, 20: 0.0036, 21: 0.089, 22: 0.1156, 23: 0.0451})

### O_A)_b) DT Tuning

In [69]:
from pyspark.sql import functions as F
g=train_over.groupBy('Working_Weekend').agg(F.count('Working_Weekend').alias('Count'))
g.sort('Count').show()

+---------------+------+
|Working_Weekend| Count|
+---------------+------+
|              0|205925|
|              1|206651|
+---------------+------+



In [70]:
g=testData.groupBy('Working_Weekend').agg(F.count('Working_Weekend').alias('Count'))
g.sort('Count').show()

+---------------+-----+
|Working_Weekend|Count|
+---------------+-----+
|              1|26203|
|              0|87796|
+---------------+-----+



In [71]:
trainingData_w = train_over.withColumn("weights2", when(trainingData["Working_Weekend"] == 0, 1.0)
    .when(trainingData["Working_Weekend"] == 1, 411881 / 205956)  # Calcolo del peso per la classe 1
)

In [72]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier

dt_param2 = DecisionTreeClassifier(labelCol="Working_Weekend", featuresCol="features",
                                  maxDepth=30,
                                  maxBins=32,
                                weightCol='weights2'  
                                  )
                                   

dt_param2 = dt_param2.fit(trainingData_w)
predictions_param2 = dt_param2.transform(testData)
print_metrics_and_cf(predictions_param2)

Accuracy : 0.7478223493188537
Test Error = 0.252178
----------------------------
Auc_: 0.6676623259810995 
 ----------------------
Precision for class  1 : 87.21%
Recall for class  1 : 78.82%
avg_F1-Score for class  1 : 82.80%
----------------------------
Precision for class  2 : 46.33%
Recall for class  2 : 61.25%
avg_F1-Score for class  2 : 52.75%
----------------------------
----------------------
avg_F1-Score: 67.78%
----------------------
[[69201. 18595.]
 [10153. 16050.]]
----------------------


In [73]:
dt_param2.featureImportances

SparseVector(24, {0: 0.0204, 1: 0.1369, 2: 0.1226, 3: 0.117, 4: 0.1316, 5: 0.1268, 6: 0.0327, 7: 0.0885, 8: 0.008, 9: 0.0022, 10: 0.0002, 11: 0.0058, 12: 0.0003, 13: 0.0091, 14: 0.0004, 15: 0.0017, 17: 0.003, 18: 0.0054, 19: 0.0001, 20: 0.0063, 21: 0.078, 22: 0.0824, 23: 0.0206})

### O_B) Ranodom Forest Base

In [74]:
# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="Working_Weekend", featuresCol="features", numTrees=10)


# Train model.  This also runs the indexers.
model = rf.fit(train_over)

# Make predictions.
predictions = model.transform(testData)
print_metrics_and_cf(predictions)

Accuracy : 0.5674435740664392
Test Error = 0.432556
----------------------------
Auc_: 0.5656856532750435 
 ----------------------
Precision for class  1 : 83.54%
Recall for class  1 : 54.59%
avg_F1-Score for class  1 : 66.03%
----------------------------
Precision for class  2 : 29.60%
Recall for class  2 : 63.96%
avg_F1-Score for class  2 : 40.47%
----------------------------
----------------------
avg_F1-Score: 53.25%
----------------------
[[47928. 39868.]
 [ 9443. 16760.]]
----------------------


In [75]:
model.featureImportances

SparseVector(24, {0: 0.0521, 1: 0.0187, 2: 0.0074, 3: 0.1684, 4: 0.0047, 5: 0.006, 6: 0.007, 7: 0.0061, 8: 0.0211, 9: 0.0, 11: 0.0001, 17: 0.0001, 18: 0.0001, 20: 0.0, 21: 0.0221, 22: 0.4441, 23: 0.242})

### O_C) NN

In [76]:
oversample_1_nn=trainingData_nn.filter(col('Working_Weekend') == 1)
num_campioni_da_generare = n_0
num_campioni_da_generare=(num_campioni_da_generare-n_1)/n_1 #percentuale 
#print(num_campioni_da_generare)
full_1_nn=oversample_1_nn
#print(full.count())
while num_campioni_da_generare>1:
    oversample_1_nn= oversample_1_nn.unionAll(full_1_nn)
    num_campioni_da_generare=num_campioni_da_generare-1.0
    print(num_campioni_da_generare)
df_minority_oversampled_nn = full_1_nn.sample(True, num_campioni_da_generare, seed=42)

oversample_1_nn = oversample_1_nn.unionAll(df_minority_oversampled_nn)

1.359732902319962
0.359732902319962


In [77]:
df_over_nn=oversample_1_nn.unionAll(trainingData_nn.filter(col('Working_Weekend') == 0))

In [78]:
from pyspark.sql import functions as F
g=df_over_nn.groupBy('Working_Weekend').agg(F.count('Working_Weekend').alias('Working_Weekend'))
g=g.sort('Working_Weekend')
g.show()

+---------------+---------------+
|Working_Weekend|Working_Weekend|
+---------------+---------------+
|              0|         205787|
|              1|         205767|
+---------------+---------------+



In [79]:
train_over_nn=df_over_nn

In [80]:
layers = [len(num_col),4]

# Create the Multilayer Perceptron Classifier and set its parameters
trainer = MultilayerPerceptronClassifier(
    layers=layers,
    labelCol="Working_Weekend",
    featuresCol="features",
    maxIter=100,  
    tol=1e-6,
    seed=None,
    blockSize=32,
    stepSize=0.03,  
    solver="l-bfgs",
    initialWeights=None,
    probabilityCol="probability",
    rawPredictionCol="rawPrediction"
)

# Train the model
model = trainer.fit(train_over_nn)

# Make predictions on the test set
predictions= model.transform(testData_nn)
print_metrics_and_cf(predictions)

Accuracy : 0.6249267913181059
Test Error = 0.375073
----------------------------
Auc_: 0.5468267565140011 
 ----------------------
Precision for class  1 : 80.11%
Recall for class  1 : 68.12%
avg_F1-Score for class  1 : 73.63%
----------------------------
Precision for class  2 : 29.25%
Recall for class  2 : 43.81%
avg_F1-Score for class  2 : 35.08%
----------------------------
----------------------
avg_F1-Score: 54.36%
----------------------
[[59897. 28037.]
 [14871. 11594.]]
----------------------
